#### 2. Recommendation Efficiency

In [ ]:
mae = mean_absolute_error(predicted_df['stars'], predicted_df['predicted_stars'])
print("MAE: ",mae)

MAE:  1.6655550762496887


Recommendation efficiency ở bài toán này sẽ đánh giá khả năng đề xuất của hệ thống trên tập test có thời gian đánh giá mới nhất so với các đánh giá cũ để kiểm tra xem qua khoảng thời gian nhất định biểu hiện hành vi của người dùng như thế nào tập dữ liệu. Cách đánh giá sẽ tập trung vào top N số lượng phim được đề xuất cho một user bất kỳ và ta sẽ kiểm tra xem trong top N có bao nhiêu phim thực sự đã được rating bởi người dùng.

Công thức sẽ sử dụng được gọi là Precision@k như sau:

**P = (# of top k recommendations that are relevant)/(# of items that are recommended)**


<p><img src="https://cdn.discordapp.com/attachments/895583585518620708/1101876809294950430/image.png"></img></p>

In [ ]:
test_users = list(test_df['author'].unique())

In [ ]:
user_limit = 5000
item_limit = 10

total_hit = 0
count = 0
for user in test_users[:user_limit]:
    user_rating = test_df[test_df['author'] == user]
    recommend_movies = get_recommend_movies(movie_neighborhood,final_review,user,item_limit,mode='favorite')
    valid_movies = list(user_rating[user_rating['stars'] >= 7]['imdb_id'])
    invalid_movies = list(user_rating[user_rating['stars'] < 5]['imdb_id'])
    recommend_hit = (len(list(set(recommend_movies).intersection(valid_movies))) - len(list(set(recommend_movies).intersection(invalid_movies)))) / item_limit
    # precisionK
    total_hit+=recommend_hit
    count+=1

print(total_hit/count)

0.01571999999999998


Độ chính xác của tập dữ liệu rơi vào khoảng 1.5%, rất thấp so với một mô hình có thể sử dụng được. Có nhiều lý do cho việc này như ta đã đề cập từ trước do các user có hành vi khác thường khiến mô hình bị sai lệch so với thực tế hay các phim trong tập kiểm thử không so bằng với các phim trong tập đề xuất khiến chúng bị rơi ra khỏi tập đề xuất. 

Sau khi thực hiện, nhóm để ý thấy vị trí được đề xuất trong tập đề xuất cũng rất quan trọng khi độ chính xác của mô hình sẽ được thể hiện trực tiếp nếu các bộ phim trong tập kiểm thử có các vị trí top cao trong tập đề xuất. Qua đó, chúng em tìm hiểu và thực hiện đánh giá theo công thức Average precision (AP):

<p><img src="https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Recommender-Systems-ML-Metrics-vs-Business-Metrics_27.png?resize=480%2C86&ssl=1"></img></p>

<p><img src="https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Recommender-Systems-ML-Metrics-vs-Business-Metrics_6.png?resize=792%2C229&ssl=1"></img></p>

In [ ]:
def avg_precision(recommendation_hit):
    count = 1
    hit = 0
    precision = 0
    for i in recommendation_hit:
        if i == 1:
            hit+=1
            precision+=  hit / count
        count+=1
    if hit == 0 :
        return 0
    else:
        return precision / hit

In [ ]:
user_limit = 5000
item_limit = 10
mean_avg_prec_K = 0
for user in test_users[:user_limit]:
    user_rating = test_df[test_df['author'] == user]
    recommend_movies = get_recommend_movies(movie_neighborhood,final_review,user,10,mode='favorite')
    valid_movies = list(user_rating[user_rating['stars'] >= 7]['imdb_id'])
    recommendation_hit = avg_precision([1 if i in valid_movies else 0 for i in recommend_movies])
    mean_avg_prec_K+= recommendation_hit
    #break
print(mean_avg_prec_K / user_limit)

0.05066791345427054


Độ chính xác sau khi sử dụng AP đã tăng lên 5% chứng tỏ các bộ phim trong tập kiểm thử thực sự có xuất hiện trong tập đề xuất đều đứng ở vị trí rất cao khiến độ chính xác cao hơn so với Precision@K

## Reference

https://neptune.ai/blog/recommender-systems-metrics#:~:text=Average%20reciprocal%20hit%20rank%20(ARHR,of%20the%20first%20correct%20item

https://www.geeksforgeeks.org/cosine-similarity/

https://medium.com/@aliozan_memetoglu/4-collaborative-filtering-and-knn-f997f8993256

https://link.springer.com/article/10.1007/s10479-016-2367-1

https://www.datarevenue.com/en-blog/building-a-production-ready-recommendation-system